In [1]:
import numpy as np

In [2]:
category_dict = {}
category_dict["Informatique"] = ["Tablette", "Ordinateur", "Portable"]
category_dict["Film"] = ["DVD", "TV", "Cinéma"]

In [4]:
def count(y):
    return sum([len(y.get(x, 0)) for x in y])

In [24]:
class Clients:
    def __init__(self, n_client, category_dict, desire_threshold=5):
        self.category_dict = category_dict
        self.category_list = list(category_dict.keys())
        self.n_client = n_client
        self.n_category = len(category_dict.keys())
        self.n_commodity = count(category_dict)
        self.desire_threshold_list = [desire_threshold] * n_client
        self.preference_matrix = np.zeros((n_client, self.n_commodity))
        self.desire_matrix = np.zeros((n_client, self.n_commodity))
        
        self.generate_client_distribution()
        
    def generate_client_distribution(self):
        proba_category = np.random.uniform(0, 0.9, (self.n_client, self.n_category))
        proba_commodity_list = []
        for j in range(proba_category.shape[1]):
            #print('---aaa---', np.expand_dims(proba_category[:,j],1).shape)
            proba_commodity_list.append(np.repeat(np.expand_dims(proba_category[:,j],1), len(self.category_dict[self.category_list[j]]), axis=1))
        self.preference_matrix = np.hstack(proba_commodity_list)
        #print(proba_commodity_list[0].shape)
        #print(proba_commodity_list[1].shape)
        #print(self.proba_commodity.shape)
        self.preference_matrix += np.random.uniform(0, 0.1, (self.n_client, self.n_commodity))
        return
    
    def get_clients(self, n):
        return np.random.choice(list(range(self.n_client)), n, replace=False)
    
    def update_desire(self, clients_list, rate_rise=None, rate_decay=0.01):
        if rate_decay is not None:
            self.desire_matrix *= 1 - rate_decay
        
        if rate_rise is None:
            rate_rise = np.random.normal()
        if rate_rise < 0:
            rate_rise = 0
        self.desire_matrix[clients_list] += rate_rise * self.preference_matrix[clients_list]
    
    def get_waiting_list(self, client):
        client_desire = self.desire_matrix[client]
        return client_desire[client_desire>self.desire_threshold_list[client]]
    
    def return_command(self, client, recommandation):
        pass

In [25]:
C = Clients(n_client=10, category_dict=category_dict)

In [26]:
C.get_clients(5)

array([0, 3, 1, 2, 6])

In [28]:
C.desire_matrix

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [18]:
C.generate_client_distribution()

In [81]:
C.update_desire(C.get_clients(1), rate_rise=0.1)

In [82]:
C.desire_matrix

array([[0.03503334, 0.03406123, 0.03200806, 0.06156058, 0.06872026,
        0.06539613],
       [0.24312447, 0.24551222, 0.28812782, 0.61330701, 0.53840514,
        0.58843675],
       [0.07995713, 0.08296628, 0.09003697, 0.14529365, 0.13972012,
        0.12823477],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.15771098, 0.15710164, 0.14919789, 0.02707551, 0.03139111,
        0.01663223],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.23370533, 0.1732092 , 0.14725276, 0.21176037, 0.2078271 ,
        0.31404906],
       [0.35450815, 0.3890847 , 0.44162286, 0.16031986, 0.13195712,
        0.20311869],
       [0.03097729, 0.03450309, 0.03253343, 0.04515095, 0.04483162,
        0.04331868]])

In [86]:
C.get_waiting_list(0)

array([], dtype=float64)

In [85]:
np.array([[1], [1,2]])

array([list([1]), list([1, 2])], dtype=object)